<a href="https://colab.research.google.com/github/techakilan/python-datascience-training/blob/master/PythonBasicDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

### CSV from Google drive

##### Mount from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##### Import csv file from google drive and create dataframe

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/colabs_data/titanic/train.csv')
train_df

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/colabs_data/titanic/test.csv')
test_df

##### head()

In [ ]:
train_df.head()

In [ ]:
train_df.head(10)

##### tail()

In [ ]:
train_df.tail()

In [ ]:
train_df.tail(10)

##### shape()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

##### Get info() of the Dataframe

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

38% of the passengers survived

#### Missing data

In [ ]:
# missing values as absolute number

train_df.isnull().sum()

In [ ]:
# missing values as percentages

missingvals = train_df.isnull().sum()/len(train_df)
missingvals.sort_values(ascending=False)

In [ ]:
train_df.nunique()

##### Age

In [ ]:
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].dropna().median())

In [ ]:
# check number of missing values
train_df['Age'].isna().sum()

##### Embarked

In [ ]:
train_df["Embarked"].value_counts()

In [ ]:
train_df["Embarked"] = train_df["Embarked"].fillna('S')

##### Cabin

In [ ]:
train_df['CabinM'] = train_df['Cabin'].fillna("U0")

In [ ]:
train_df['CabinM']

In [ ]:
missingvals = train_df.isnull().sum()/len(train_df)
missingvals.sort_values(ascending=False)

#### Data exploration

In [ ]:
train_df.columns

###### Seperate features into numerical and categorical

In [ ]:
train_df_num = train_df[['Age','SibSp','Parch','Fare']]
train_df_cat = train_df[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
for i in train_df_num.columns:
    plt.hist(train_df_num[i])
    plt.title(i)
    plt.show()

Age is close to normal. Others are scattered.

In [ ]:
sns.heatmap(train_df_num.corr())

The only significant correlation is between SibSp and Parch

In [ ]:
pd.pivot_table(train_df, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

##### Survived

In [ ]:
for i in train_df_cat.columns:
    sns.barplot(x=train_df_cat[i].value_counts().index,y=train_df_cat[i].value_counts()).set_title(i)
    plt.show()

Ticket and Cabin data is messy

Majority of the people travelled 3rd class

There were approx double the number of males compared to females

Embarked: Most of the passengers boarded the ship from Southampton.

In [ ]:
print(pd.pivot_table(train_df, index = 'Survived', columns = 'Pclass',
                     values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(train_df, index = 'Survived', columns = 'Sex', 
                     values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(train_df, index = 'Survived', columns = 'Embarked', 
                     values = 'Ticket' ,aggfunc ='count'))

More people survived in the first class

More women surviced compared to males

Survived to Non survivor ratio is better for those embarked on Cherboug

### Feature engineering

##### Cabin

In [ ]:
[x for x in train_df.CabinM if len(x.split(' '))>1]

In [ ]:
train_df['cabin_multiple'] = train_df.Cabin.apply(lambda x: 0 if pd.isna(x) 
                                                    else len(x.split(' ')))
train_df['cabin_multiple'].value_counts()



In [ ]:
pd.pivot_table(train_df, index = 'Survived', columns = 'cabin_multiple',
               values = 'Ticket' ,aggfunc ='count')

In [ ]:
train_df['cabin_first_letter'] = train_df.Cabin.apply(lambda x: str(x)[0])
#comparing survival rates by cabin
print(train_df.cabin_first_letter.value_counts())
pt_table = pd.pivot_table(train_df,index='Survived',columns='cabin_first_letter', 
                        values = 'Name', aggfunc='count')
lbl = [x for x in pt_table]
survivedvals = [pt_table[x][0] for x in pt_table]
pct = [pt_table[x][1]/(pt_table[x][0]+pt_table[x][1]) for x in pt_table]

pd.DataFrame(pct, lbl)

We decide to drop Cabin

In [ ]:
train_df = train_df.drop(["Cabin", "CabinM","cabin_multiple","cabin_first_letter"], axis=1)

In [ ]:
train_df

##### Ticket

In [ ]:
train_df['numeric_ticket'] = train_df.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
train_df['ticket_letters'] = train_df.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1])
                                            .replace('.','').replace('/','')
                                            .lower() if len(x.split(' ')[:-1]) >0 else 0)

In [ ]:
train_df['numeric_ticket']

In [ ]:
train_df['ticket_letters']

In [ ]:
train_df = train_df.drop(["Ticket", "ticket_letters"], axis=1)

##### Name

In [ ]:
train_df['name_title'] = train_df.Name.apply(lambda x: x.split(',')[1]
                                                        .split('.')[0].strip())
train_df['name_title'].value_counts()

In [ ]:
train_df = train_df.drop(["Name"], axis=1)

##### Parch SibSp

In [ ]:
train_df['Family'] = train_df.Parch + train_df.SibSp

In [ ]:
train_df['Is_Alone'] = train_df.Family == 0

In [ ]:
train_df = train_df.drop(["Family","Parch","SibSp"], axis=1)

In [ ]:
train_df.loc[0]

##### Encoding

In [ ]:
from sklearn import preprocessing

In [ ]:
# Encode Age
train_df['Sex'] = preprocessing.LabelEncoder().fit_transform(train_df['Sex'])


In [ ]:
train_df['Sex'] 

In [ ]:
# Encode Embarked
train_df  = pd.get_dummies(train_df, columns=["Embarked"], prefix="Emb", drop_first = True)

In [ ]:
train_df.loc[0]

In [ ]:
# Encode Name title
train_df = pd.get_dummies(train_df, columns=["name_title"], prefix="Name", drop_first = True)

In [ ]:
# Encode Is_Alone

